## VAD

In [12]:
import json

from nemo.collections.asr.models.label_models import EncDecSpeakerLabelModel
from nemo.collections.asr.models.msdd_models import ClusteringDiarizer

from omegaconf import OmegaConf


# Load yaml file
with open("./config/nemo/diar_infer_telephonic.yaml") as f:
    cfg = OmegaConf.load(f)

meta = {
    "audio_filepath":  "mono_file.wav",
    "offset": 0,
    "duration": None,
    "label": "infer",
    "text": "-",
    "rttm_filepath": None,
    "uem_filepath": None,
}

manifest_path = "infer_manifest.json"
with open("infer_manifest.json", "w") as fp:
    json.dump(meta, fp)
    fp.write("\n")

cfg.diarizer.manifest_filepath = str(manifest_path)
cfg.diarizer.out_dir = "infer_out_dir"

speaker_model = EncDecSpeakerLabelModel.from_pretrained(
    model_name="titanet_large", map_location=None
)
speaker_params = {
    "window_length_in_sec": [1.5, 1.25, 1.0, 0.75, 0.5],
    "shift_length_in_sec": [0.75, 0.625, 0.5, 0.375, 0.25],
    "multiscale_weights": [1, 1, 1, 1, 1],
    "save_embeddings": True,
}
cluster_params = {
    "oracle_num_speakers": False,
    "max_num_speakers": 8,
    "enhanced_count_thres": 80,
    "max_rp_threshold": 0.25,
    "sparse_search_volume": 30,
    "maj_vote_spk_count": False,
}

clus_diar_model = ClusteringDiarizer(cfg=cfg, speaker_model=speaker_model)

[NeMo I 2023-07-25 09:06:34 cloud:58] Found existing object /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2023-07-25 09:06:34 cloud:64] Re-using file from: /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2023-07-25 09:06:34 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-07-25 09:06:35 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-07-25 09:06:35 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2023-07-25 09:06:35 features:291] PADDING: 16
[NeMo I 2023-07-25 09:06:35 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2023-07-25 09:06:35 clustering_diarizer:127] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2023-07-25 09:06:35 cloud:58] Found existing object /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2023-07-25 09:06:35 cloud:64] Re-using file from: /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2023-07-25 09:06:35 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-07-25 09:06:35 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

[NeMo I 2023-07-25 09:06:35 features:291] PADDING: 16
[NeMo I 2023-07-25 09:06:35 save_restore_connector:249] Model EncDecClassificationModel was successfully restored from /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.


In [16]:
import librosa
import soundfile as sf

filepath = "./mono_file.mp3"
waveform, sample_rate = librosa.load(filepath, sr=None)
sf.write("./mono_file.wav", waveform, sample_rate, "PCM_16")

In [17]:
clus_diar_model.diarize()

[NeMo W 2023-07-25 09:37:44 clustering_diarizer:411] Deleting previous clustering diarizer outputs.


[NeMo I 2023-07-25 09:37:44 speaker_utils:93] Number of files to diarize: 1
[NeMo I 2023-07-25 09:37:44 clustering_diarizer:309] Split long audio file to avoid CUDA memory issue


splitting manifest: 100%|██████████| 1/1 [00:00<00:00, 11.75it/s]

[NeMo I 2023-07-25 09:37:44 vad_utils:101] The prepared manifest file exists. Overwriting!
[NeMo I 2023-07-25 09:37:44 classification_models:268] Perform streaming frame-level VAD
[NeMo I 2023-07-25 09:37:44 collections:298] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2023-07-25 09:37:44 collections:299] Dataset loaded with 3 items, total duration of  0.04 hours.
[NeMo I 2023-07-25 09:37:44 collections:301] # 3 files loaded accounting to # 1 labels


[NeMo I 2023-07-25 09:37:46 clustering_diarizer:250] Generating predictions with overlapping input segments


[NeMo I 2023-07-25 09:37:47 clustering_diarizer:262] Converting frame level prediction to speech/no-speech segment in start and end times format.


creating speech segments: 100%|██████████| 1/1 [00:00<00:00,  3.86it/s]

[NeMo I 2023-07-25 09:37:47 clustering_diarizer:287] Subsegmentation for embedding extraction: scale0, infer_out_dir/speaker_outputs/subsegments_scale0.json
[NeMo I 2023-07-25 09:37:47 clustering_diarizer:343] Extracting embeddings for Diarization
[NeMo I 2023-07-25 09:37:47 collections:298] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2023-07-25 09:37:47 collections:299] Dataset loaded with 104 items, total duration of  0.04 hours.
[NeMo I 2023-07-25 09:37:47 collections:301] # 104 files loaded accounting to # 1 labels


[NeMo I 2023-07-25 09:37:48 clustering_diarizer:389] Saved embedding files to infer_out_dir/speaker_outputs/embeddings
[NeMo I 2023-07-25 09:37:48 clustering_diarizer:287] Subsegmentation for embedding extraction: scale1, infer_out_dir/speaker_outputs/subsegments_scale1.json
[NeMo I 2023-07-25 09:37:48 clustering_diarizer:343] Extracting embeddings for Diarization
[NeMo I 2023-07-25 09:37:48 collections:298] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2023-07-25 09:37:48 collections:299] Dataset loaded with 132 items, total duration of  0.04 hours.
[NeMo I 2023-07-25 09:37:48 collections:301] # 132 files loaded accounting to # 1 labels
[NeMo I 2023-07-25 09:37:48 clustering_diarizer:389] Saved embedding files to infer_out_dir/speaker_outputs/embeddings
[NeMo I 2023-07-25 09:37:48 clustering_diarizer:287] Subsegmentation for embedding extraction: scale2, infer_out_dir/speaker_outputs/subsegments_scale2.json
[NeMo I 2023-07-25 09:37:48 clustering_diarizer:343] Extrac

In [13]:
from typing import List, Optional, Tuple, Union

import torch
import torchaudio
from faster_whisper.vad import VadOptions, get_speech_timestamps


class VadService:
    """VAD Service for audio files."""

    def __init__(self) -> None:
        """Initialize the VAD Service."""
        self.sample_rate = 16000
        self.options = VadOptions(
            threshold=0.5,
            min_speech_duration_ms=250,
            max_speech_duration_s=30,
            min_silence_duration_ms=100,
            window_size_samples=512,
            speech_pad_ms=30,
        )

    def __call__(
        self, waveform: torch.Tensor, group_timestamps: Optional[bool] = True
    ) -> Tuple[Union[List[dict], List[List[dict]]], torch.Tensor]:
        """
        Use the VAD model to get the speech timestamps. Dual channel pipeline.

        Args:
            waveform (torch.Tensor): Audio tensor.
            group_timestamps (Optional[bool], optional): Group timestamps. Defaults to True.

        Returns:
            Tuple[Union[List[dict], List[List[dict]]], torch.Tensor]: Speech timestamps and audio tensor.
        """
        if waveform.size(0) == 1:
            waveform = waveform.squeeze(0)

        speech_timestamps = get_speech_timestamps(
            audio=waveform, vad_options=self.options
        )

        _speech_timestamps_list = [
            {"start": ts["start"], "end": ts["end"]} for ts in speech_timestamps
        ]

        if group_timestamps:
            speech_timestamps_list = self.group_timestamps(_speech_timestamps_list)
        else:
            speech_timestamps_list = _speech_timestamps_list

        return speech_timestamps_list, waveform

    def group_timestamps(
        self, timestamps: List[dict], threshold: Optional[float] = 3.0
    ) -> List[List[dict]]:
        """
        Group timestamps based on a threshold.

        Args:
            timestamps (List[dict]): List of timestamps.
            threshold (float, optional): Threshold to use for grouping. Defaults to 3.0.

        Returns:
            List[List[dict]]: List of grouped timestamps.
        """
        grouped_segments = [[]]

        for i in range(len(timestamps)):
            if (
                i > 0
                and (timestamps[i]["start"] - timestamps[i - 1]["end"]) > threshold
            ):
                grouped_segments.append([])

            grouped_segments[-1].append(timestamps[i])

        return grouped_segments

    def save_audio(self, filepath: str, audio: torch.Tensor) -> None:
        """
        Save audio tensor to file.

        Args:
            filepath (str): Path to save the audio file.
            audio (torch.Tensor): Audio tensor.
        """
        torchaudio.save(
            filepath, audio.unsqueeze(0), self.sample_rate, bits_per_sample=16
        )

def read_audio(filepath: str, sample_rate: int = 16000) -> Tuple[torch.Tensor, float]:
    """
    Read an audio file and return the audio tensor.

    Args:
        filepath (str): Path to the audio file.
        sample_rate (int): The sample rate of the audio file. Defaults to 16000.

    Returns:
        Tuple[torch.Tensor, float]: The audio tensor and the audio duration.
    """
    wav, sr = torchaudio.load(filepath)

    if wav.size(0) > 1:
        wav = wav.mean(dim=0, keepdim=True)

    if sr != sample_rate:
        transform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate)
        wav = transform(wav)
        sr = sample_rate

    audio_duration = float(wav.shape[1]) / sample_rate

    return wav.squeeze(0), audio_duration

def sr2s(v: int) -> float:
    """
    Convert milliseconds to seconds.

    Args:
        v (int): Value in milliseconds.

    Returns:
        float: Value in seconds.
    """
    return v / 16000

In [51]:
waveform, _ = read_audio("./mono_file.wav")

vad_service = VadService()

speech_ts, _ = vad_service(waveform, True)
for ts in speech_ts:
    _ts = ts[0]
    print(
        f"Start: {sr2s(_ts['start'])}, End: {sr2s(_ts['end'])}"
    )

42
Start: 12.514, End: 12.99
Start: 13.25, End: 14.366
Start: 15.042, End: 16.862
Start: 17.762, End: 18.814
Start: 19.426, End: 20.766
Start: 21.666, End: 24.83
Start: 26.178, End: 29.886
Start: 30.786, End: 33.022
Start: 34.146, End: 37.214
Start: 38.338, End: 40.318
Start: 41.218, End: 42.782
Start: 43.682, End: 44.318
Start: 45.73, End: 46.494
Start: 47.778, End: 50.366
Start: 51.106, End: 52.926
Start: 53.954, End: 55.582
Start: 55.682, End: 56.926
Start: 57.954, End: 60.286
Start: 61.154, End: 64.254
Start: 65.026, End: 67.614
Start: 68.418, End: 68.99
Start: 69.922, End: 71.55
Start: 72.578, End: 75.838
Start: 76.61, End: 77.918
Start: 78.562, End: 79.454
Start: 79.746, End: 81.086
Start: 82.05, End: 83.902
Start: 84.738, End: 86.462
Start: 87.586, End: 90.782
Start: 91.746, End: 96.542
Start: 97.73, End: 98.27
Start: 99.586, End: 100.03
Start: 100.162, End: 100.862
Start: 101.794, End: 103.454
Start: 104.898, End: 107.486
Start: 108.226, End: 109.854
Start: 114.274, End: 114.75

## Segmentation

In [3]:
window_length_in_sec = [1.5, 1.25, 1.0, 0.75, 0.5] # Window length(s) in sec (floating-point number). either a number or a list. ex) 1.5 or [1.5,1.0,0.5]
shift_length_in_sec = [0.75, 0.625, 0.5, 0.375, 0.25] # Shift length(s) in sec (floating-point number). either a number or a list. ex) 0.75 or [0.75,0.5,0.25]
multiscale_weights = [1, 1, 1, 1, 1]

In [4]:
window_lengths_in_sec = window_length_in_sec
shift_lengths_in_sec = shift_length_in_sec

window_lengths, shift_lengths, multiscale_weights = (
    window_lengths_in_sec,
    shift_lengths_in_sec,
    multiscale_weights,
)

multiscale_args_dict = {'use_single_scale_clustering': False}
scale_dict = {k: (w, s) for k, (w, s) in enumerate(zip(window_lengths, shift_lengths))}

In [8]:
import math

def get_subsegments(segment_start: float, segment_end: float, window: float, shift: float) -> List[List[float]]:
    """
    Return a list of subsegments based on the segment start and end time and the window and shift length.

    Args:
        segment_start (float): Segment start time.
        segment_end (float): Segment end time.
        window (float): Window length.
        shift (float): Shift length.

    Returns:
        List[List[float]]: List of subsegments with start time and duration.
    """
    start = segment_start
    duration = segment_end - segment_start
    base = math.ceil((duration - window) / shift)
    
    subsegments: List[List[float]] = []
    slices = 1 if base < 0 else base + 1
    for slice_id in range(slices):
        end = start + window

        if end > segment_end:
            end = segment_end

        subsegments.append([start, end - start])

        start = segment_start + (slice_id + 1) * shift

    return subsegments

In [11]:
def _run_segmentation(
    vad_outputs: List[dict],
    window: float,
    shift: float,
    min_subsegment_duration: float = 0.05,
) -> List[dict]:
    """"""
    scale_segment = []
    for segment in vad_outputs:
        segment_start, segment_end = sr2s(segment["start"]), sr2s(segment["end"])
        subsegments = get_subsegments(segment_start, segment_end, window, shift)

        for subsegment in subsegments:
            start, duration = subsegment
            if duration > min_subsegment_duration:
                scale_segment.append({"offset": start, "duration": duration})

    return scale_segment

In [9]:
from nemo.collections.asr.models import EncDecSpeakerLabelModel

from torch.cuda.amp import autocast
from torch.utils.data import Dataset

speaker_model = EncDecSpeakerLabelModel.from_pretrained(
    model_name="titanet_large", map_location=None
)

def collate_fn(batch: List[Tuple[torch.Tensor, torch.Tensor]]):
    """"""
    _, audio_lengths = zip(*batch)

    has_audio = audio_lengths[0] is not None
    fixed_length = int(max(audio_lengths))

    audio_signal, new_audio_lengths = [], []
    for sig, sig_len in batch:
        if has_audio:
            sig_len = sig_len.item()
            chunck_len = sig_len - fixed_length

            if chunck_len < 0:
                repeat = fixed_length // sig_len
                rem = fixed_length % sig_len
                sub = sig[-rem:] if rem > 0 else torch.tensor([])
                rep_sig = torch.cat(repeat * [sig])
                sig = torch.cat((rep_sig, sub))
            new_audio_lengths.append(torch.tensor(fixed_length))

            audio_signal.append(sig)

    if has_audio:
        audio_signal = torch.stack(audio_signal)
        audio_lengths = torch.stack(new_audio_lengths)
    else:
        audio_signal, audio_lengths = None, None

    return audio_signal, audio_lengths

class AudioSegmentDataset(Dataset):
    def __init__(self, waveform: torch.Tensor, segments: List[dict], sample_rate=16000) -> None:
        self.waveform = waveform
        self.segments = segments
        self.sample_rate = sample_rate

    def __len__(self) -> int:
        return len(self.segments)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        segment_info = self.segments[idx]
        offset_samples = int(segment_info["offset"] * self.sample_rate)
        duration_samples = int(segment_info["duration"] * self.sample_rate)

        segment = self.waveform[offset_samples:offset_samples + duration_samples]

        return segment, torch.tensor(segment.shape[0]).long()

def _extract_embeddings(waveform: torch.Tensor, scale_segments: List[dict]):
    """
    This method extracts speaker embeddings from segments passed through manifest_file
    Optionally you may save the intermediate speaker embeddings for debugging or any use. 
    """
    all_embs = torch.empty([0])

    dataset = AudioSegmentDataset(waveform, scale_segments)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=64, shuffle=False, collate_fn=collate_fn
    )

    for batch in dataloader:
        _batch = [x.to(speaker_model.device) for x in batch]
        audio_signal, audio_signal_len = _batch

        with autocast():
            _, embeddings = speaker_model.forward(input_signal=audio_signal, input_signal_length=audio_signal_len)
            embeddings = embeddings.view(-1, embeddings.shape[-1])
            all_embs = torch.cat((all_embs, embeddings.cpu().detach()), dim=0)
        del _batch, audio_signal, audio_signal_len, embeddings

    embeddings, time_stamps = [], []
    for i, segment in enumerate(scale_segments):
        if i == 0:
            embeddings = all_embs[i].view(1, -1)
        else:
            embeddings = torch.cat((embeddings, all_embs[i].view(1, -1)))

        time_stamps.append([segment['offset'], segment['duration']])

    return embeddings, time_stamps

[NeMo I 2023-07-27 16:43:31 cloud:58] Found existing object /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2023-07-27 16:43:31 cloud:64] Re-using file from: /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2023-07-27 16:43:31 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-07-27 16:43:34 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-07-27 16:43:34 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2023-07-27 16:43:34 features:291] PADDING: 16
[NeMo I 2023-07-27 16:43:35 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.


## Clustering

In [23]:
import copy

from nemo.collections.asr.parts.utils.offline_clustering import SpeakerClustering
from pyannote.core import Annotation, Segment


def get_contiguous_stamps(stamps):
    """
    Return contiguous time stamps
    """
    lines = copy.deepcopy(stamps)
    contiguous_stamps = []
    for i in range(len(lines) - 1):
        start, end, speaker = lines[i].split()
        next_start, next_end, next_speaker = lines[i + 1].split()
        if float(end) > float(next_start):
            avg = str((float(next_start) + float(end)) / 2.0)
            lines[i + 1] = ' '.join([avg, next_end, next_speaker])
            contiguous_stamps.append(start + " " + avg + " " + speaker)
        else:
            contiguous_stamps.append(start + " " + end + " " + speaker)
    start, end, speaker = lines[-1].split()
    contiguous_stamps.append(start + " " + end + " " + speaker)
    return contiguous_stamps


def merge_stamps(lines):
    """
    Merge time stamps of the same speaker.
    """
    stamps = copy.deepcopy(lines)
    overlap_stamps = []
    for i in range(len(stamps) - 1):
        start, end, speaker = stamps[i].split()
        next_start, next_end, next_speaker = stamps[i + 1].split()
        if float(end) == float(next_start) and speaker == next_speaker:
            stamps[i + 1] = ' '.join([start, next_end, next_speaker])
        else:
            overlap_stamps.append(start + " " + end + " " + speaker)

    start, end, speaker = stamps[-1].split()
    overlap_stamps.append(start + " " + end + " " + speaker)

    return overlap_stamps


def labels_to_pyannote_object(labels):
    """
    Convert the given labels to pyannote object to calculate DER and for visualization
    """
    annotation = Annotation()

    for label in labels:
        start, end, speaker = label.strip().split()
        start, end = float(start), float(end)
        annotation[Segment(start, end)] = speaker

    return annotation


def perform_clustering(embs_and_timestamps, clustering_params):
    """
    Performs spectral clustering on embeddings with time stamps generated from VAD output

    Args:
        embs_and_timestamps (dict): This dictionary contains the following items indexed by unique IDs.
            'embeddings' : Tensor containing embeddings. Dimensions:(# of embs) x (emb. dimension)
            'timestamps' : Tensor containing ime stamps list for each audio recording
            'multiscale_segment_counts' : Tensor containing the number of segments for each scale
        clustering_params (dict): clustering parameters provided through config that contains max_num_speakers (int),

    Returns:
        all_reference (list[uniq_name,Annotation]): reference annotations for score calculation
        all_hypothesis (list[uniq_name,Annotation]): hypothesis annotations for score calculation

    """
    speaker_clustering = SpeakerClustering(cuda=True)

    base_scale_idx = embs_and_timestamps["multiscale_segment_counts"].shape[0] - 1
    cluster_labels = speaker_clustering.forward_infer(
        embeddings_in_scales=embs_and_timestamps["embeddings"],
        timestamps_in_scales=embs_and_timestamps["timestamps"],
        multiscale_segment_counts=embs_and_timestamps["multiscale_segment_counts"],
        multiscale_weights=embs_and_timestamps["multiscale_weights"],
        oracle_num_speakers=-1,
        max_num_speakers=int(clustering_params["max_num_speakers"]),
        max_rp_threshold=float(clustering_params["max_rp_threshold"]),
        sparse_search_volume=int(clustering_params["sparse_search_volume"]),
    )

    del embs_and_timestamps
    torch.cuda.empty_cache()

    timestamps = speaker_clustering.timestamps_in_scales[base_scale_idx]
    cluster_labels = cluster_labels.cpu().numpy()
    if len(cluster_labels) != timestamps.shape[0]:
        raise ValueError("Mismatch of length between cluster_labels and timestamps.")

    lines = []
    for idx, label in enumerate(cluster_labels):
        tag = "speaker_" + str(label)
        stt, end = timestamps[idx]
        lines.append(f"{stt} {end} {tag}")

    cont_lines = get_contiguous_stamps(lines)
    labels = merge_stamps(cont_lines)

    return labels

## Final Loop

In [37]:
import torch
from typing import List

def get_argmin_mat(timestamps_in_scales: List[torch.Tensor]) -> List[torch.Tensor]:
    """
    Calculate the mapping between the base scale and other scales. A segment from a longer scale is
    repeatedly mapped to a segment from a shorter scale or the base scale.

    Args:
        timestamps_in_scales (list):
            List containing timestamp tensors for each scale.
            Each tensor has dimensions of (Number of base segments) x 2.

    Returns:
        session_scale_mapping_list (list):
            List containing argmin arrays indexed by scale index.
    """
    scale_list = list(range(len(timestamps_in_scales)))
    segment_anchor_list = [torch.mean(timestamps_in_scales[scale_idx], dim=1) for scale_idx in scale_list]

    base_scale_idx = max(scale_list)
    base_scale_anchor = segment_anchor_list[base_scale_idx]
    base_scale_anchor = base_scale_anchor.view(-1, 1)

    session_scale_mapping_list = []
    for scale_idx in scale_list:
        curr_scale_anchor = segment_anchor_list[scale_idx].view(1, -1)
        distance = torch.abs(curr_scale_anchor - base_scale_anchor)
        argmin_mat = torch.argmin(distance, dim=1)
        session_scale_mapping_list.append(argmin_mat)

    return session_scale_mapping_list


# Check https://github.com/NVIDIA/NeMo/blob/2cc09425aba3e9b3cfdba43a3188eaef58227055/nemo/collections/asr/models/msdd_models.py#L756
def get_cluster_avg_embs(
    emb_scale_seq_dict: dict,
    clustering_labels: list,
    session_scale_mapping_list: list,
    scale_n: int,
):
    """
    MSDD requires cluster-average speaker embedding vectors for each scale. This function calculates an average embedding vector for each cluster (speaker)
    and each scale.

    Args:
        emb_scale_seq_dict (dict):
            Dictionary containing embedding sequence for each scale. Keys are scale index in integer.
        clus_labels (list):
            Clustering results from clustering diarizer including all the sessions provided in input manifest files.
        session_scale_mapping_dict (list):
            List containing argmin arrays indexed by scale index.

    Returns:
        emb_sess_avg_dict (dict):
            Dictionary containing speaker mapping information and cluster-average speaker embedding vector.
            Each session-level dictionary is indexed by scale index in integer.
        output_clus_label_dict (dict):
            Subegmentation timestamps in float type and Clustering result in integer type. Indexed by `uniq_id` keys.
    """
    emb_sess_avg_dict = {
        scale_index: {key: [] for key in emb_scale_seq_dict[scale_n - 1].keys()}
        for scale_index in emb_scale_seq_dict.keys()
    }
    output_clus_label_dict, emb_dim = self.get_base_clus_label_dict(clus_labels, emb_scale_seq_dict)
    all_scale_clus_label_dict = self.assign_labels_to_longer_segs(
        output_clus_label_dict, session_scale_mapping_dict
    )
    for scale_index in emb_scale_seq_dict.keys():
        for uniq_id, _emb_tensor in emb_scale_seq_dict[scale_index].items():
            if type(_emb_tensor) == list:
                emb_tensor = torch.tensor(np.array(_emb_tensor))
            else:
                emb_tensor = _emb_tensor
            clus_label_list = all_scale_clus_label_dict[scale_index][uniq_id]
            spk_set = set(clus_label_list)

            # Create a label array which identifies clustering result for each segment.
            label_array = torch.Tensor(clus_label_list)
            avg_embs = torch.zeros(emb_dim, self.max_num_speakers)
            for spk_idx in spk_set:
                selected_embs = emb_tensor[label_array == spk_idx]
                avg_embs[:, spk_idx] = torch.mean(selected_embs, dim=0)

            if speaker_mapping_dict is not None:
                inv_map = {clus_key: rttm_key for rttm_key, clus_key in speaker_mapping_dict[uniq_id].items()}
            else:
                inv_map = None

            emb_sess_avg_dict[scale_index][uniq_id] = {'mapping': inv_map, 'avg_embs': avg_embs}
    return emb_sess_avg_dict, output_clus_label_dict


In [42]:
window_lengths, shift_lengths, multiscale_weights = (
    [1.5, 1.25, 1.0, 0.75, 0.5],
    [0.75, 0.625, 0.5, 0.375, 0.25],
    [1, 1, 1, 1, 1],
)
scale_dict = {k: (w, s) for k, (w, s) in enumerate(zip(window_lengths, shift_lengths))}

# VAD
waveform, _ = read_audio("./mono_file.wav")
vad_service = VadService()

vad_outputs, _ = vad_service(waveform, False)

# Segmentation
all_embeddings, all_timestamps, all_segment_indexes = [], [], []

scales = scale_dict.items()
for scale_idx, (window, shift) in scales:
    scale_segments = _run_segmentation(vad_outputs, window, shift)

    _embeddings, _timestamps = _extract_embeddings(waveform, scale_segments)

    if len(_embeddings) != len(_timestamps):
        raise ValueError("Mismatch of counts between embedding vectors and timestamps")

    all_embeddings.append(_embeddings)
    all_segment_indexes.append(_embeddings.shape[0])
    all_timestamps.append(torch.tensor(_timestamps))

multiscale_embeddings_and_timestamps = {
    "embeddings": torch.cat(all_embeddings, dim=0),
    "timestamps": torch.cat(all_timestamps, dim=0),
    "multiscale_segment_counts": torch.tensor(all_segment_indexes),
    "multiscale_weights": torch.tensor([1, 1, 1, 1, 1]).unsqueeze(0).float(),
}

# Clustering
clustering_params = dict(
    oracle_num_speakers=False,
    max_num_speakers=8,
    enhanced_count_thres=80,
    max_rp_threshold=0.25,
    sparse_search_volume=30,
    maj_vote_spk_count=False,
)
clustering_labels = perform_clustering(
    embs_and_timestamps=multiscale_embeddings_and_timestamps,
    clustering_params=clustering_params,
)

split_index = multiscale_embeddings_and_timestamps["multiscale_segment_counts"].tolist()
embeddings_in_scales = list(torch.split(
    multiscale_embeddings_and_timestamps["embeddings"], split_index, dim=0
))
timestamps_in_scales = list(torch.split(
    multiscale_embeddings_and_timestamps["timestamps"], split_index, dim=0
))
session_scale_mapping_list = get_argmin_mat(timestamps_in_scales)

scale_mapping_argmat, emb_scale_seq_dict = {}, {}
for scale_idx in range(len(session_scale_mapping_list)):
    mapping_argmat = session_scale_mapping_list[scale_idx]
    scale_mapping_argmat[scale_idx] = mapping_argmat

    emb_scale_seq_dict[scale_idx] = embeddings_in_scales[scale_idx]

emb_sess_avg_dict, base_clus_label_dict = get_cluster_avg_embs(
    emb_scale_seq_dict, clustering_labels, session_scale_mapping_list, len(scale_dict)
)
# emb_scale_seq_dict['session_scale_mapping'] = session_scale_mapping_dict
# return emb_sess_avg_dict, emb_scale_seq_dict, base_clus_label_dict, metric

['12.513999938964844 0.47600001096725464 speaker_1', '13.25 0.5 speaker_1', '13.5 0.5 speaker_1', '13.75 0.5 speaker_1', '14.0 0.3659999966621399 speaker_1', '15.041999816894531 0.5 speaker_1', '15.291999816894531 0.5 speaker_1', '15.541999816894531 0.5 speaker_1', '15.791999816894531 0.5 speaker_1', '16.04199981689453 0.5 speaker_1', '16.29199981689453 0.5 speaker_1', '16.54199981689453 0.3199999928474426 speaker_1', '17.761999130249023 0.5 speaker_0', '18.011999130249023 0.5 speaker_0', '18.261999130249023 0.5 speaker_0', '18.511999130249023 0.3019999861717224 speaker_0', '19.426000595092773 0.5 speaker_0', '19.676000595092773 0.5 speaker_0', '19.926000595092773 0.5 speaker_0', '20.176000595092773 0.5 speaker_0', '20.426000595092773 0.3400000035762787 speaker_0', '21.666000366210938 0.5 speaker_0', '21.916000366210938 0.5 speaker_0', '22.166000366210938 0.5 speaker_0', '22.416000366210938 0.5 speaker_0', '22.666000366210938 0.5 speaker_0', '22.916000366210938 0.5 speaker_0', '23.1660